In [ ]:
%use plotly
import kotlinx.coroutines.*
import java.time.Duration
import java.time.Instant

val data = runBlocking { readAllUData("./data/log.anon") + readAllUData("./data/log2.anon") }

In [ ]:
fun plot(udata: List<UData>, title: String): Plot {
    val splitByAlgo = udata.groupBy { ud ->
        toSigningAlgorithm(ud.algo)
    }.toList()
    val total = udata.size
    val percentages = splitByAlgo.map { (_, udl) -> udl.size }
    val alogrithm = splitByAlgo.map { (alg, udl) ->
        val percentage = udl.size.toDouble() / total * 100
        val progressString = "%.2f".format(percentage)
        val algName = when (alg) {
            SigningAlgorithm.ECDSA_P_256 -> "ECDSA P-256"
            SigningAlgorithm.ECDSA_P_384 -> "ECDSA P-384"
            else -> alg.toString()
        }
        "$algName: $progressString%"
    }

    return Plotly.plot {
        pie {
            values(percentages)
            labels(alogrithm)
            textinfo = TextInfo.`label`
        }
        layout {
            width = 500
            height = 375
            title {
                text = title
            }
        }
    }
}

In [ ]:
plot(data, "Percentage of validations per signing algorithm")

In [ ]:
plot(
    data.filter { u -> u.domainName.size <= 2 && u.sub == false },
    "Percentage of validations per algorithm for root zone and TLDs"
)

In [ ]:
plot(
    data.filter { u -> u.domainName.size <= 2 && u.sub == false }.distinctBy { u -> Pair(u.domainName, u.algo) },
    "Percentage of validations per algorithm for root zone and TLDs without duplicate algorithm and subdomain"
)

In [ ]:
plot(
    data.filter { u -> u.sub == true },
    "Percentage of validations per algorithm for subdomains of prefixes"
)

In [ ]:
val splitPerTLD =
    data.groupBy { ud -> ud.domainName.dropLast(1).lastOrNull() }.mapValues { (_, v) -> v.size }.toList()
        .sortedByDescending { (_, v) -> v }

val topAmount = 5
val top = splitPerTLD.take(topAmount)
val other = splitPerTLD.drop(topAmount).map { (k, v) -> v }.sum()

Plotly.plot {
    pie {
        values(top.map { (_, v) -> v } + other)
        labels(top.map { (tld, _) ->
            if (tld == null) {
                "root"
            } else {
                tld
            }
        } + "other")
        textinfo = TextInfo.`label+text+percent`
    }
    layout {
        title = "Percentage of validations for TLDs and root zone including subdomains"
        width = 500
        height = 375
    }
}